# NPI Extraction

## 1. Dependencies and Data Loading

## 2. JSON to DataFrame for NPI

## 3. NPI API Calls

## 4. Data Join and final export

In [26]:
#import findspark
#findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,BooleanType,DoubleType
from pyspark.sql import functions as F

In [19]:
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("PySpark Read JSON") \
    .getOrCreate()

In [66]:
file = 'network_rate_sample.json'
# Reading multiline json file
multi_df = spark.read.option("multiline","true") \
                      .json(file)

In [67]:
multi_df.printSchema()

root
 |-- in_network: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- billing_code: string (nullable = true)
 |    |    |-- billing_code_type: string (nullable = true)
 |    |    |-- billing_code_type_version: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- negotiated_rates: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- negotiated_prices: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |    |-- additional_information: string (nullable = true)
 |    |    |    |    |    |    |-- billing_class: string (nullable = true)
 |    |    |    |    |    |    |-- billing_code_modifier: array (nullable = true)
 |    |    |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |    |    |-- expiration_date: strin

In [59]:
new_df = multi_df.withColumn("providers",F.explode(F.arrays_zip("provider_references.provider_group_id", 
                                                "provider_references.provider_groups")))\
                 .select("providers.provider_references.provider_group_id",
                         "providers.provider_references.provider_groups")

AnalysisException: 'No such struct field provider_references in 0, 1;'

In [58]:
df.withColumn("price_product", F.explode(F.arrays_zip("Price", "Product")))\
    .select("price_product.Price", "price_product.Product")\
    .show()

NameError: name 'df' is not defined

In [56]:
new_df.show()

+--------------------+
|                 col|
+--------------------+
|[0, [[[1609076330...|
|[1, [[[1447420815...|
|[2, [[[1629005558...|
|[3, [[[1083672067...|
|[4, [[[1740661461...|
|[5, [[[1588284459...|
|[6, [[[1780748939...|
|[7, [[[1487639381...|
|[8, [[[1699747303...|
|[9, [[[1457323164...|
|[10, [[[128597548...|
|[11, [[[188173803...|
|[12, [[[134666421...|
|[13, [[[178062052...|
|[14, [[[135634886...|
|[15, [[[128503357...|
|[16, [[[173026653...|
|[17, [[[174060101...|
|[18, [[[148798351...|
|[19, [[[112444953...|
+--------------------+
only showing top 20 rows



In [60]:
from pyspark.sql.functions import col, explode_outer, from_json, lit, concat
from pyspark.sql.types import StructType, ArrayType


def get_json_df(input_df, primary_partition_column, json_column_name, spark_session):
    '''
    Description:
    This function provides the schema of json records and the dataframe to be used for flattening
    :param input_df: [type: pyspark.sql.dataframe.DataFrame] input dataframe
    :primary_partition_column: [type: string] name of primary partition column
    :param json_column_name: [type: string] name of the column with json string
    :param spark_session: SparkSession object
    :return df: dataframe to be used for flattening
    '''
    input_df = input_df if primary_partition_column is None else input_df.drop(primary_partition_column)
    # creating a column transformedJSON to create an outer struct
    df1 = input_df.withColumn('transformed_json', concat(lit("""{"transformedJSON" :"""), input_df[json_column_name], lit("""}""")))
    json_df = spark_session.read.json(df1.rdd.map(lambda row: row.transformed_json))
    # get schema
    json_schema = json_df.schema
    df = df1.drop(json_column_name).withColumn(json_column_name, from_json(col('transformed_json'), json_schema)).drop('transformed_json').select(f'{json_column_name}.*', '*').drop(json_column_name)
    return df


def execute_autoflatten(df, json_column_name):
    '''
    Description:
    This function executes the core autoflattening operation
    :param df: [type: pyspark.sql.dataframe.DataFrame] dataframe to be used for flattening
    :param json_column_name: [type: string] name of the column with json string
    :return df: DataFrame containing flattened records
    '''
    # gets all fields of StructType or ArrayType in the nested_fields dictionary
    nested_fields = dict([
        (field.name, field.dataType)
        for field in df.schema.fields
        if isinstance(field.dataType, ArrayType) or isinstance(field.dataType, StructType)
    ])

    # repeat until all nested_fields i.e. belonging to StructType or ArrayType are covered
    while nested_fields:
        # if there are any elements in the nested_fields dictionary
        if nested_fields:
            # get a column
            column_name = list(nested_fields.keys())[0]
            # if field belongs to a StructType, all child fields inside it are accessed
            # and are aliased with complete path to every child field
            if isinstance(nested_fields[column_name], StructType):
                unnested = [col(column_name + '.' + child).alias(column_name + '>' + child) for child in [ n.name for n in  nested_fields[column_name]]]
                df = df.select("*", *unnested).drop(column_name)
            # else, if the field belongs to an ArrayType, an explode_outer is done
            elif isinstance(nested_fields[column_name], ArrayType):
                df = df.withColumn(column_name, explode_outer(column_name))

        # Now that df is updated, gets all fields of StructType and ArrayType in a fresh nested_fields dictionary
        nested_fields = dict([
            (field.name, field.dataType)
            for field in df.schema.fields
            if isinstance(field.dataType, ArrayType) or isinstance(field.dataType, StructType)
        ])

    # renaming all fields extracted with json> to retain complete path to the field
    for df_col_name in df.columns:
        df = df.withColumnRenamed(df_col_name, df_col_name.replace("transformedJSON", json_column_name))
    return df



def execute(input_df, primary_partition_column, json_column_name, spark_session):
    '''
    Description:
    This function executes the flattening of json records in the dataframe
    :param input_df: [type: pyspark.sql.dataframe.DataFrame] input dataframe
    :primary_partition_column: [type: string] name of primary partition column
    :param json_column_name: [type: string] name of the column with json string
    :param spark_session: SparkSession object
    :return unstd_df: contains flattened dataframe with unstandardized column name format
    '''
    json_df = get_json_df(input_df, primary_partition_column, json_column_name, spark_session)
    unstd_df = execute_autoflatten(json_df, json_column_name)
    return unstd_df

In [84]:
cln_df = execute(multi_df,"provider_references","provider_references",spark_session = spark)

AnalysisException: 'Cannot resolve column name "provider_group_id" among (in_network, last_updated_on, reporting_entity_name, reporting_entity_type, version);'

In [99]:
n_row_df = multi_df.select(
    "provider_references",
    F.explode("provider_references").alias("provider_references_explode")
).select("provider_references", "provider_references_explode.*")

In [107]:
n_row_df.provider_references

+--------------------+-----------------+--------------------+
| provider_references|provider_group_id|     provider_groups|
+--------------------+-----------------+--------------------+
|[[0, [[[160907633...|                0|[[[1609076330], [...|
|[[0, [[[160907633...|                1|[[[1447420815], [...|
|[[0, [[[160907633...|                2|[[[1629005558], [...|
|[[0, [[[160907633...|                3|[[[1083672067, 18...|
|[[0, [[[160907633...|                4|[[[1740661461], [...|
+--------------------+-----------------+--------------------+
only showing top 5 rows



In [104]:
m_row_df = n_row_df.select(
    "provider_groups","provider_group_id",
    F.explode("provider_groups").alias("provider_groups_explode")
).select("provider_groups","provider_group_id", "provider_groups_explode.*")

In [105]:
m_row_df.show()

+--------------------+-----------------+--------------------+----------------+
|     provider_groups|provider_group_id|                 npi|             tin|
+--------------------+-----------------+--------------------+----------------+
|[[[1609076330], [...|                0|        [1609076330]|[ein, 204768243]|
|[[[1447420815], [...|                1|        [1447420815]|[ein, 261999831]|
|[[[1629005558], [...|                2|        [1629005558]|[ein, 205096637]|
|[[[1083672067, 18...|                3|[1083672067, 1821...|[ein, 200319618]|
|[[[1083672067, 18...|                3|[1306800552, 1801...|[ein, 752674367]|
|[[[1740661461], [...|                4|        [1740661461]|[ein, 474302355]|
|[[[1588284459, 12...|                5|[1588284459, 1265...|[ein, 621617225]|
|[[[1780748939], [...|                6|        [1780748939]|[ein, 593493196]|
|[[[1780748939], [...|                6|        [1457396376]|[ein, 651118475]|
|[[[1780748939], [...|                6|        [157